In [9]:
# Importing necessary libraries
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Reading the data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Dropping non-beneficial columns
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()


Optimizer: SGD
  Units: 16, Layers: 2
Epoch 1/25
7/7 [==============================] - 0s 5ms/step - loss: 0.2464 - accuracy: 0.6012
Epoch 2/25
7/7 [==============================] - 0s 5ms/step - loss: 0.2448 - accuracy: 0.6015
Epoch 3/25
7/7 [==============================] - 0s 5ms/step - loss: 0.2433 - accuracy: 0.6015
Epoch 4/25
7/7 [==============================] - 0s 4ms/step - loss: 0.2419 - accuracy: 0.6012
Epoch 5/25
7/7 [==============================] - 0s 4ms/step - loss: 0.2405 - accuracy: 0.6049
Epoch 6/25
7/7 [==============================] - 0s 5ms/step - loss: 0.2392 - accuracy: 0.6067
Epoch 7/25
7/7 [==============================] - 0s 5ms/step - loss: 0.2380 - accuracy: 0.6069
Epoch 8/25
7/7 [==============================] - 0s 5ms/step - loss: 0.2368 - accuracy: 0.6072
Epoch 9/25
7/7 [==============================] - 0s 5ms/step - loss: 0.2357 - accuracy: 0.6101
Epoch 10/25
7/7 [==============================] - 0s 4ms/step - loss: 0.2346 - accuracy: 0.6103
E

In [ ]:

# Reducing categorical variables
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_counts[application_counts < 500].index.tolist()
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_types_to_replace = classification_counts[classification_counts < 1000].index.tolist()
for cls in classification_types_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")


In [ ]:
# One-hot encoding
application_df = pd.get_dummies(application_df)

# Splitting the data into features and target
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns=['IS_SUCCESSFUL']).values

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Defining model creation function
def create_model(units, layers):
    nn = tf.keras.Sequential()
    nn.add(tf.keras.layers.Dense(units=256, activation='relu', input_dim=X_train.shape[1]))
    for _ in range(layers - 1):
        nn.add(tf.keras.layers.Dense(units=units, activation='relu'))
    nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    return nn


In [ ]:
# Defining combinations of units and layers
units_values = [16, 64, 128]
layers_values = [2, 3, 4]
combinations = [(units, layers) for units in units_values for layers in layers_values]



In [ ]:
# Custom loss function
def custom_loss_function(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=y_pred.dtype)
    return tf.reduce_mean(tf.square(y_true - y_pred))



In [ ]:
# Optimizers
Optimizers = {
    'SGD': tf.keras.optimizers.SGD,
    'RMSprop': tf.keras.optimizers.RMSprop,
    'Adagrad': tf.keras.optimizers.Adagrad,
    'Adadelta': tf.keras.optimizers.Adadelta,
    'Adam': tf.keras.optimizers.Adam,
    'Adamax': tf.keras.optimizers.Adamax,
    'Nadam': tf.keras.optimizers.Nadam,
    'Ftrl': tf.keras.optimizers.Ftrl
}

# Create an empty list to store results
results = []
with tf.keras.utils.custom_object_scope({'custom_loss_function': custom_loss_function}):
    # Loop through each optimizer
    for optimizer_name, optimizer_class in Optimizers.items():
        print(f"Optimizer: {optimizer_name}")
        # results = {}  # Dictionary to store results for this optimizer
        
        # Loop through each combination of units and layers
        for units, layers in combinations:
            print(f"  Units: {units}, Layers: {layers}")
            
            # Create and compile the model
            model = create_model(units, layers)
            model.compile(loss='custom_loss_function', optimizer=optimizer_class(), metrics=['accuracy'])
            
            # Fit the model
            model.fit(X_train_scaled, y_train, epochs=25, batch_size=4096, verbose=1)
            
            # Evaluate the model
            _, accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)
            
            # Store the results for this combination
            results.append({'Optimizer': optimizer_name, 'Units': units, 'Layers': layers, 'Accuracy': accuracy})

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(results)

In [12]:
len(results_df)

72

In [13]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Optimizer  72 non-null     object 
 1   Units      72 non-null     int64  
 2   Layers     72 non-null     int64  
 3   Accuracy   72 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.4+ KB


In [14]:
results_df_sorted = results_df.sort_values(by=['Accuracy', 'Layers', 'Units'], ascending=[False, True, True])
display(results_df_sorted.head(50))
display(results_df_sorted.tail(len(results_df_sorted)-50))

,Optimizer,Units,Layers,Accuracy
44,Adam,128,4,0.730758
13,RMSprop,64,3,0.730029
57,Nadam,64,2,0.729300
54,Nadam,16,2,0.729155
55,Nadam,16,3,0.728863
53,Adamax,128,4,0.728863
40,Adam,64,3,0.728571
56,Nadam,16,4,0.728571
39,Adam,64,2,0.728426
50,Adamax,64,4,0.728426


,Optimizer,Units,Layers,Accuracy
20,Adagrad,16,4,0.565889
23,Adagrad,64,4,0.564286
25,Adagrad,128,3,0.554082
34,Adadelta,128,3,0.550000
29,Adadelta,16,4,0.534402
18,Adagrad,16,2,0.534111
63,Ftrl,16,2,0.534111
66,Ftrl,64,2,0.534111
69,Ftrl,128,2,0.534111
64,Ftrl,16,3,0.534111


In [15]:
# Select the best model (top row after sorting by accuracy)
best_model_row = results_df_sorted.iloc[0]

# Extract units and layers from the best model row
best_units = best_model_row['Units']
best_layers = best_model_row['Layers']
best_model = create_model(best_units, best_layers)
best_model.compile(loss='binary_crossentropy', optimizer=best_model_row['Optimizer'], metrics=['accuracy'])
# Fit the best model
best_model.fit(X_train_scaled, y_train, epochs=50, batch_size=4096, verbose=1)

# Evaluate the best model
_, best_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=1)

# Save the best model
best_model.save('AlphabetSoupCharity_Optimization.h5')

# Print the accuracy of the best model
print(f"Accuracy of the best model: {best_accuracy}")

Epoch 1/50
7/7 [==============================] - 1s 11ms/step - loss: 0.6697 - accuracy: 0.6080
Epoch 2/50
7/7 [==============================] - 0s 10ms/step - loss: 0.5915 - accuracy: 0.7227
Epoch 3/50
7/7 [==============================] - 0s 10ms/step - loss: 0.5682 - accuracy: 0.7285
Epoch 4/50
7/7 [==============================] - 0s 11ms/step - loss: 0.5561 - accuracy: 0.7303
Epoch 5/50
7/7 [==============================] - 0s 11ms/step - loss: 0.5525 - accuracy: 0.7313
Epoch 6/50
7/7 [==============================] - 0s 10ms/step - loss: 0.5492 - accuracy: 0.7329
Epoch 7/50
7/7 [==============================] - 0s 10ms/step - loss: 0.5474 - accuracy: 0.7333
Epoch 8/50
7/7 [==============================] - 0s 10ms/step - loss: 0.5461 - accuracy: 0.7336
Epoch 9/50
7/7 [==============================] - 0s 10ms/step - loss: 0.5446 - accuracy: 0.7347
Epoch 10/50
7/7 [==============================] - 0s 10ms/step - loss: 0.5438 - accuracy: 0.7340
Epoch 11/50
7/7 [============

In [16]:
best_model_row

Optimizer        Adam
Units             128
Layers              4
Accuracy     0.730758
Name: 44, dtype: object